In [2]:
import math
import pandas as pd
import altair as alt
import numpy as np
import flammkuchen as fl
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import sys,os,os.path
sys.path.append(os.path.expanduser('../src'))

from spinorama.graph import graph_params_default, graph_freq, graph_spinorama, contour_params_default, graph_radar
from spinorama.load import filter_graphs, load_normalize

df_all = fl.load('../cache.parse_all_speakers.h5')

In [8]:
# speaker = 'JBL 308P Mark ii'
speaker = 'Genelec 8341A'

df_klippel = df_all[speaker]['ASR']['asr']
spl_H = df_klippel['SPL Horizontal_unmelted']
spl_V = df_klippel['SPL Vertical_unmelted']
df_computed = load_normalize(filter_graphs('Adam S2V', spl_H, spl_V))

df_klippel_eq = df_all['Adam S2V']['ASR']['asr']
spl_H_eq = df_klippel_eq['SPL Horizontal_unmelted']
spl_V_eq = df_klippel_eq['SPL Vertical_unmelted']
df_computed_eq = load_normalize(filter_graphs('Adam S2V', spl_H_eq, spl_V_eq))

def a2v(angle):
    if angle == 'Freq':
            return -1000
    elif angle == 'On Axis':
            return 0
    iangle = int(angle[:-1])
    return iangle

def transform(spl):
    db_max = spl['On Axis'].max()
    df = spl.reindex(columns=sorted(spl.columns, key=lambda a: a2v(a)))-db_max
    # x,y,z
    x = df.Freq
    y = [a2v(i) for i in df.loc[:, df.columns != 'Freq'].columns]
    z = df.loc[:, df.columns != 'Freq'].T.values
    # color
    # c = np.where(z<-21, -72, np.multiply(np.floor_divide(z,3),3))
    c = np.multiply(np.floor_divide(z,3),3)
    return x, y, z, c

In [9]:
fig = make_subplots(rows=1, cols=2, 
                    specs=[[{'type': 'surface'}, {'type': 'surface'}]],
                    subplot_titles=['Horizontal SPL', 'Vertical SPL'])
x_h, y_h, z_h, c_h = transform(spl_H)
x_v, y_v, z_v, c_v = transform(spl_V)
colors  = [[0.0, '#5c77a5'], 
           [0.05, '#dc842a'], 
           [0.1, '#c85857'], 
           [0.15, '#89b5b1'], 
           [0.2, '#71a152'], 
           [0.25, '#bab0ac'], 
           [0.3, '#e15759'], 
           [0.35, '#b07aa1'], 
           [0.4, '#76b7b2'], 
           [1, '#ff9da7']]
trace_h = go.Surface(
    z=z_h, x=x_h, y=y_h, 
    name='Horizontal SPL', 
    showscale=False,
    #colorscale=colors,
    surfacecolor=c_h,
    cmin=-72, cmax=5)
trace_v = go.Surface(
    z=z_v, x=x_v, y=y_v, 
    name='Vertical SPL', 
    showscale=True,
    #colorscale=colors,
    surfacecolor=c_v,
    cmin=-72, cmax=5)
fig.add_trace(trace_h, row=1, col=1)
fig.add_trace(trace_v, row=1, col=2)
fig.update_layout(
    title='Speakers {0}'.format(speaker), 
    xaxis_title='Frequency (Hz)',
    autosize=False, 
    width=1000, height=500,
    margin=dict(l=20, r=20, b=20, t=75),
    scene = dict(
        xaxis = dict(type='log', range=[1+math.log(2), 3+math.log(2)]),
        yaxis = dict(range=[-180, 180]),
        zaxis = dict(range=[-45,5]),
    #camera = dict(
    #    up=dict(x=0, y=0, z=0),
    #    center=dict(x=0, y=0, z=0),
    #    eye=dict(x=1.25, y=1.25, z=1.25)
    #),
))
fig.update_traces(
    contours_z=dict(show=True,  project_z=True))
fig.show()